In [3]:
import pandas as pd
from itertools import combinations

In [4]:
nvalues = pd.read_csv('../data/nutritional_values.csv')
recipes = pd.read_csv('../data/filtered_recipes_100.csv',sep='|')

In [5]:
df = pd.merge(recipes, nvalues, on='Recipe_id', how='right')

In [6]:
wanted_cols =['Recipe_id','Categoria','energia','grasa','proteina','carbohidratos']
df = df[wanted_cols].head()
cc = list(combinations(df.Recipe_id,2)) #antes de esto hacer filtradosde comida y cena etc
df.set_index('Recipe_id',inplace=True)
df

,Categoria,energia,grasa,proteina,carbohidratos
Recipe_id,,,,,
1,Cócteles y bebidas,35.090000,0.052000,0.188000,2.232000
5,Recetas de Verduras,103.252526,5.622834,1.402017,10.663666
8,Cócteles y bebidas,44.937500,0.373438,2.306250,7.909375
10,Cócteles y bebidas,601.000000,3.170000,30.550000,110.300000
27,Recetas de Pescado,1658.207052,153.715161,12.688162,59.213363


In [7]:
cc_cols = ['Recipe1','Recipe2'] #poner comida/cena whatever

combined_df = pd.DataFrame(columns = cc_cols)
combined_df['Recipe1'] = [c[0] for c in cc]
combined_df['Recipe2'] = [c[1] for c in cc]
combined_df

,Recipe1,Recipe2
0,1,5
1,1,8
2,1,10
3,1,27
4,5,8
5,5,10
6,5,27
7,8,10
8,8,27
9,10,27


In [8]:
macros = ['energia','grasa','proteina','carbohidratos']
for macro in macros:
    combined_df[macro] = [df[macro][c[1]]+df[macro][c[0]] for c in cc]
combined_df

,Recipe1,Recipe2,energia,grasa,proteina,carbohidratos
0,1,5,138.342526,5.674834,1.590017,12.895666
1,1,8,80.027500,0.425438,2.494250,10.141375
2,1,10,636.090000,3.222000,30.738000,112.532000
3,1,27,1693.297052,153.767161,12.876162,61.445363
4,5,8,148.190026,5.996272,3.708267,18.573041
5,5,10,704.252526,8.792834,31.952017,120.963666
6,5,27,1761.459578,159.337995,14.090179,69.877029
7,8,10,645.937500,3.543438,32.856250,118.209375
8,8,27,1703.144552,154.088599,14.994412,67.122738
9,10,27,2259.207052,156.885161,43.238162,169.513363


In [9]:
goals = { #por ejemplo, estos valores sería lo que falta después de fijar desayunos etc
    'goal_energia': 2000,
    'goal_grasa': 10,
    'goal_proteina': 20,
    'goal_carbohidratos': 30
}

#func para extraer los errores
for macro in macros:
    macro_error = macro + '_error'
    macro_goal = 'goal_' + macro
    combined_df[macro_error] = abs(goals[macro_goal]-combined_df[macro])

# error total
macro_errors = [macro+'_error' for macro in macros]
combined_df['total_error'] = combined_df[macro_errors].sum(axis=1) #los sumo pero podría ser otra operación 

combined_df

,Recipe1,Recipe2,energia,grasa,proteina,carbohidratos,energia_error,grasa_error,proteina_error,carbohidratos_error,total_error
0,1,5,138.342526,5.674834,1.590017,12.895666,1861.657474,4.325166,18.409983,17.104334,1901.496957
1,1,8,80.027500,0.425438,2.494250,10.141375,1919.972500,9.574562,17.505750,19.858625,1966.911437
2,1,10,636.090000,3.222000,30.738000,112.532000,1363.910000,6.778000,10.738000,82.532000,1463.958000
3,1,27,1693.297052,153.767161,12.876162,61.445363,306.702948,143.767161,7.123838,31.445363,489.039310
4,5,8,148.190026,5.996272,3.708267,18.573041,1851.809974,4.003728,16.291733,11.426959,1883.532394
5,5,10,704.252526,8.792834,31.952017,120.963666,1295.747474,1.207166,11.952017,90.963666,1399.870323
6,5,27,1761.459578,159.337995,14.090179,69.877029,238.540422,149.337995,5.909821,39.877029,433.665267
7,8,10,645.937500,3.543438,32.856250,118.209375,1354.062500,6.456562,12.856250,88.209375,1461.584687
8,8,27,1703.144552,154.088599,14.994412,67.122738,296.855448,144.088599,5.005588,37.122738,483.072373
9,10,27,2259.207052,156.885161,43.238162,169.513363,259.207052,146.885161,23.238162,139.513363,568.843738


In [17]:
def getClosestSum(n, df):
    return df.nsmallest(n, 'total_error')

In [18]:
getClosestSum(1, combined_df)

,Recipe1,Recipe2,energia,grasa,proteina,carbohidratos,energia_error,grasa_error,proteina_error,carbohidratos_error,total_error
6,5,27,1761.459578,159.337995,14.090179,69.877029,238.540422,149.337995,5.909821,39.877029,433.665267


In [19]:
getClosestSum(3, combined_df) #a partir de aquí extraer los ids, meterlos en una lista para pasarlos todos

,Recipe1,Recipe2,energia,grasa,proteina,carbohidratos,energia_error,grasa_error,proteina_error,carbohidratos_error,total_error
6,5,27,1761.459578,159.337995,14.090179,69.877029,238.540422,149.337995,5.909821,39.877029,433.665267
8,8,27,1703.144552,154.088599,14.994412,67.122738,296.855448,144.088599,5.005588,37.122738,483.072373
3,1,27,1693.297052,153.767161,12.876162,61.445363,306.702948,143.767161,7.123838,31.445363,489.039310
